In [5]:
import os
os.chdir("C:\\Users\\tsarcevic\\PycharmProjects\\fingerprinting-toolbox")
import warnings
warnings.filterwarnings('ignore')

In [7]:
import pickle
import numpy as np
import pandas as pd
import collections

from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

from scipy.stats import chi2
from imblearn.under_sampling import *

from datasets import GermanCredit
from attacks import *
from parameter_guidelines.guidelines import *

In [8]:
# original data
data = Nursery().preprocessed()
X = data.drop('target', axis=1).to_numpy()
y = data['target']

In [9]:
# fingerprinted data
gamma=1
fp_dir = "parameter_guidelines/fingerprinted_data/nursery/attr_subset_8/"
fingerprinted_data = pd.read_csv(fp_dir + "universal_g{}_x1_l32_u1_sk0.csv".format(gamma))
fp_data = Nursery().preprocessed(fp_data=fingerprinted_data)
#data = data.to_numpy()
X_fp = fp_data.drop('target', axis=1)#.to_numpy()
y_fp = fp_data['target']

In [10]:
# baseline
score_baseline = fp_cross_val_score(MLPClassifier(random_state=9), X, y, X_fp, y_fp, scoring='accuracy')['test_score']
score_baseline

array([0.78780864, 0.74421296, 0.79552469, 0.72867619, 0.7047472 ])

In [11]:
# this function will serve as a pipeline for cross validation attacked dataset
def attack_cross_val_score(X, y, target, attack, attack_strength, n_folds=5):
    accuracy = []
    for fold in range(n_folds):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=fold, shuffle=True)
        train = pd.concat([X_train, y_train], axis=1)
        attacked_train = attack.run(train, attack_strength, random_state=fold)
        attacked_X = attacked_train.drop(target, axis=1)
        attacked_y = attacked_train[target]
        model = MLPClassifier()
        model.fit(attacked_X, attacked_y)
        acc = accuracy_score(y_test, model.predict(X_test))
        accuracy.append(acc)
    return accuracy

In [12]:
# random attack
attack_utility_scores = attack_cross_val_score(X_fp, y_fp, 'target', HorizontalSubsetAttack(), 0.7)
attack_utility_scores

[0.8271604938271605,
 0.8271604938271605,
 0.841820987654321,
 0.8209876543209876,
 0.8233024691358025]

### Targeted attack

In [13]:
def targeted_attack_cross_val_score(X, y, attack_strength, n_folds=5, sampler_version=1, sampler_n_neighbors=3):
    accuracy = []
    for fold in range(n_folds):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=fold, shuffle=True)
        train = pd.concat([X_train, y_train], axis=1)
        sampler = NearMiss(version=sampler_version, 
                           sampling_strategy=strategy(attack_strength, y_train), 
                           n_neighbors=sampler_n_neighbors)
        #sampler = InstanceHardnessThreshold(estimator=GradientBoostingClassifier(),
        #                                    sampling_strategy=strategy(attack_strength, y_train)) -- some internal error
        attacked_X, attacked_y = sampler.fit_resample(X_train, y_train)
        model = MLPClassifier()
        model.fit(attacked_X, attacked_y)
        acc = accuracy_score(y_test, model.predict(X_test))
        accuracy.append(acc)
    return accuracy

In [14]:
def strategy(attack_strength, y):
    strategy = y.value_counts() * (1-attack_strength)
    strategy = {key: round(x) for key, x in strategy.items()}
    return strategy

In [15]:
# targeted attack
attack_utility_scores = targeted_attack_cross_val_score(X_fp, y_fp, 0.7)
attack_utility_scores
np.mean(attack_utility_scores) # version 1 ###### the best (and the only one actually better than random)

0.6749228395061728

In [16]:
# experimenting with the number of neighbors
res = dict()
for n in range(1, 10):
    # targeted attack
    attack_utility_scores = targeted_attack_cross_val_score(X_fp, y_fp, 0.7, sampler_n_neighbors=n) # 3 shows to be the best (default)
    res[n] = np.mean(attack_utility_scores) # version 1 ###### the best (and the only one actually better than random)
print('version 1')
print(res)
print(max(res.values()))

version 1
{1: 0.7416666666666666, 2: 0.7162808641975309, 3: 0.6739197530864198, 4: 0.6619598765432099, 5: 0.6429783950617283, 6: 0.6366512345679013, 7: 0.6292438271604939, 8: 0.6227623456790123, 9: 0.6175925925925926}
0.7416666666666666


In [17]:
# targeted attack
attack_utility_scores = targeted_attack_cross_val_score(X_fp, y_fp, 0.7, sampler_version=2)
attack_utility_scores
np.mean(attack_utility_scores) # version 2

0.719212962962963

In [18]:
# targeted attack
attack_utility_scores = targeted_attack_cross_val_score(X_fp, y_fp, 0.7, sampler_version=3)
np.mean(attack_utility_scores) # version 3

0.6344135802469135

Here update the targeted_attack_cross_val_score with the best strategy.

In [19]:
with open('parameter_guidelines/Nursery/evaluation/robustness_horizontal_universal_c95_e30.pickle', 'rb') as infile:
    robustness = pickle.load(infile)

In [20]:
robustness

{1: 0.9,
 1.11: 0.9,
 1.25: 0.9,
 1.43: 0.9,
 1.67: 0.9,
 2: 0.9,
 2.5: 0.9,
 3: 0.9,
 4: 0.9,
 5: 0.9,
 10: 0.8,
 18: 0.6}

In [21]:
fpattr=8
n_exp = 10
results = {gamma: [] for gamma in robustness}
for gamma, attack_strength in robustness.items():
    print(gamma, attack_strength)
    for exp in range(n_exp):
        # take one fp ds
        fp_dir = "parameter_guidelines/fingerprinted_data/nursery/attr_subset_{}/".format(fpattr)
        fingerprinted_data = pd.read_csv(fp_dir + "universal_g{}_x1_l32_u1_sk{}.csv".format(gamma, exp))
        fp_data = Nursery().preprocessed(fp_data=fingerprinted_data)
        #data = data.to_numpy()
        X_fp = fp_data.drop('target', axis=1)#.to_numpy()
        y_fp = fp_data['target']
        
        # calc baseline fp utility
        score_baseline = fp_cross_val_score(MLPClassifier(random_state=9), X, y, X_fp, y_fp, scoring='accuracy')['test_score']

        # calc attacked 
        attack_utility_scores = targeted_attack_cross_val_score(X_fp, y_fp, attack_strength, 
                                                                sampler_version=1, sampler_n_neighbors=1) # CHANGE HERE

        # take relative loss
        rel_loss = (score_baseline - attack_utility_scores) / score_baseline
        results[gamma].append(rel_loss)

1 0.9
1.11 0.9
1.25 0.9
1.43 0.9
1.67 0.9
2 0.9
2.5 0.9
3 0.9
4 0.9
5 0.9
10 0.8
18 0.6


In [22]:
results

{1: [array([ 0.125857  ,  0.08553655,  0.13870029,  0.02632918, -0.02917563]),
  array([0.16523929, 0.15079365, 0.15549077, 0.12626315, 0.12219876]),
  array([0.13246362, 0.10638298, 0.14690841, 0.0520806 , 0.12248034]),
  array([0.18722574, 0.05687974, 0.16960457, 0.09819751, 0.10643731]),
  array([0.16402116, 0.03161398, 0.17903734, 0.11717746, 0.04322641]),
  array([0.22677866, 0.17080745, 0.24603534, 0.15223976, 0.1347692 ]),
  array([0.15560061, 0.08351293, 0.16286645, 0.07233606, 0.02963732]),
  array([0.16068796, 0.07176532, 0.20046948, 0.1349596 , 0.00782852]),
  array([0.19588123, 0.07302772, 0.1470726 , 0.05424077, 0.06059681]),
  array([0.17706334, 0.07555089, 0.17633302, 0.10998144, 0.00599216])],
 1.11: [array([ 0.11232063,  0.06059009,  0.12237094,  0.02217431, -0.04608215]),
  array([0.14314214, 0.10907184, 0.1387422 , 0.12454841, 0.05316716]),
  array([0.17241379, 0.12583806, 0.17896679, 0.06666457, 0.15013237]),
  array([0.12940608, 0.0427027 , 0.13837375, 0.02147015, 

In [23]:
with open('parameter_guidelines/Nursery/evaluation/rel_undersampling_attack_utilities_mlp_fpattr{}_e{}.pkl'.format(fpattr, n_exp), 'wb') as outfile:
    pickle.dump(results, outfile)